In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dataframe = pd.read_csv('credit_risk_dataset.csv')


In [7]:
print(dataframe.head().T)

                                   0          1         2        3        4
person_age                        22         21        25       23       24
person_income                  59000       9600      9600    65500    54400
person_home_ownership           RENT        OWN  MORTGAGE     RENT     RENT
person_emp_length              123.0        5.0       1.0      4.0      8.0
loan_intent                 PERSONAL  EDUCATION   MEDICAL  MEDICAL  MEDICAL
loan_grade                         D          B         C        C        C
loan_amnt                      35000       1000      5500    35000    35000
loan_int_rate                  16.02      11.14     12.87    15.23    14.27
loan_status                        1          0         1        1        1
loan_percent_income             0.59        0.1      0.57     0.53     0.55
cb_person_default_on_file          Y          N         N        N        Y
cb_person_cred_hist_length         3          2         3        2        4


In [8]:
print(dataframe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB
None


In [9]:
print(dataframe[['person_age', 'person_income', 'loan_amnt']].describe())

         person_age  person_income     loan_amnt
count  32581.000000   3.258100e+04  32581.000000
mean      27.734600   6.607485e+04   9589.371106
std        6.348078   6.198312e+04   6322.086646
min       20.000000   4.000000e+03    500.000000
25%       23.000000   3.850000e+04   5000.000000
50%       26.000000   5.500000e+04   8000.000000
75%       30.000000   7.920000e+04  12200.000000
max      144.000000   6.000000e+06  35000.000000


In [ ]:
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeImputer

df_iterative_imputed = imputer_iterative.fit_transform(dataframe)
scalar = StandardScaler()
scaled_data = pd.DataFrame(scalar.fit_transform(dataframe))

plt.figure(figsize=(12, 6))
sns.boxplot(x='loan_intent', y='person_emp_length', data=dataframe)
plt.title('Employment Length by Loan Intent')
plt.xlabel('Loan Intent')
plt.ylabel('Employment Length (Years)')
plt.show()

ValueError: could not convert string to float: 'RENT'